In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
# Any results you write to the current directory are saved as output.


train_data = pd.read_pickle('images_l.pkl')
train_labels = pd.read_pickle('labels_l.pkl')
test_data = pd.read_pickle('images_test.pkl')
unlabel = pd.read_pickle('images_ul.pkl')

number_train_images = int(train_data.shape[0] * 0.9)

X_train = train_data[:number_train_images]
y_train = train_labels[:number_train_images]

X_validation = train_data[number_train_images:]
y_validation = train_labels[number_train_images:]

train_unlabel = unlabel[:int(unlabel.shape[0] * 0.9)]
validation_unlabel = unlabel[int(unlabel.shape[0] * 0.9):]
print(train_unlabel.shape)
print(validation_unlabel.shape)

(27000, 56, 56)
(3000, 56, 56)


In [73]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class MNIST_Dataset(Dataset):
    
    def __init__(self, X, y, transform = transforms.Compose(
                     [
                      transforms.ToTensor()])):
            
        self.transform = transform
        # self.X = np.reshape(self.X, (len(self.X), 7, 7, 8))
        self.X = X.astype('float32')
        print(self.X.shape)
        self.y = y
        
    @classmethod
    def from_train(cls, file_path_images, file_path_labels):
        train_images = pd.read_pickle(file_path_images)
        train_labels = pd.read_csv(file_path_labels)
        
        return cls(X=train_images, y=torch.from_numpy(train_labels['Category'].values))
        
    @classmethod
    def from_test(cls, file_path_images):
        test_images = pd.read_pickle(file_path_images)
        
        return cls(X=test_images, y=None)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.transform(self.X[idx])
        # X = X.to(device)
        if self.y is not None:
            return X, self.y[idx]   #.to(device)
        else:
            return X

In [26]:
train_data = pd.read_pickle('images_l.pkl')
train_labels = pd.read_pickle('labels_l.pkl')
test_data = pd.read_pickle('images_test.pkl')
unlabel = pd.read_pickle('images_ul.pkl')

number_train_images = int(train_data.shape[0] * 0.9)

X_train = train_data[:number_train_images]
y_train = train_labels[:number_train_images]
print(X_train.shape)
print(y_train.shape)

X_validation = train_data[number_train_images:]
y_validation = train_labels[number_train_images:]

print(X_validation.shape)
print(y_validation.shape)
print(test_data.shape)

train_unlabel = unlabel[:int(unlabel.shape[0] * 0.9)]
validation_unlabel = unlabel[int(unlabel.shape[0] * 0.9):]
print(train_unlabel.shape)
print(validation_unlabel.shape)

(27000, 56, 56)
(27000, 36)
(3000, 56, 56)
(3000, 36)
(15000, 56, 56)
(27000, 56, 56)
(3000, 56, 56)


In [49]:
batch_size = 32

train_dataset = MNIST_Dataset(X_train, y_train)
validation_dataset = MNIST_Dataset(X_validation, None)
test_dataset = MNIST_Dataset.from_test('images_test.pkl')

# unlabel_train_dataset = MNIST_Dataset(train_unlabel, train_unlabel)
# unlabel_validation_dataset = MNIST_Dataset(validation_unlabel, validation_unlabel)

# Loading
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

# unlabel_train_loader = torch.utils.data.DataLoader(dataset=unlabel_train_dataset,
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# unlabel_validation_loader = torch.utils.data.DataLoader(dataset=unlabel_validation_dataset,
#                                            batch_size=batch_size, 
#                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

(27000, 56, 56, 1)
(3000, 56, 56, 1)
(15000, 56, 56, 1)


## normalize unlabeled data

In [2]:
X_train = X_train.astype('float32') / 255.
X_validation = X_train.astype('float32') / 255.
X_train = np.reshape(X_train, (len(X_train), 56, 56, 1))
X_validation = np.reshape(X_validation, (len(X_validation), 56, 56, 1))

train_unlabel = train_unlabel.astype('float32') / 255.
validation_unlabel = validation_unlabel.astype('float32') / 255.
train_unlabel = np.reshape(train_unlabel, (len(train_unlabel), 56, 56, 1))
validation_unlabel = np.reshape(validation_unlabel, (len(validation_unlabel), 56, 56, 1))
print(train_unlabel.shape)
print(validation_unlabel.shape)

(27000, 56, 56, 1)
(3000, 56, 56, 1)


In [90]:
import keras
from keras import layers

input_img = keras.Input(shape=(56, 56, 1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)        
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
# x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 8) i.e. 392-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
# x = layers.Conv2D(16, (3, 3), activation='relu')(x)
# x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

from keras.callbacks import TensorBoard
autoencoder.summary()
autoencoder.fit(X_train, X_train,
                epochs=2,
                batch_size=32,
                shuffle=True,
                validation_data=(validation_unlabel, validation_unlabel),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])


Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 56, 56, 1)]       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 56, 56, 16)        160       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 28, 28, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 28, 28, 8)         1160      
                                                                 
 conv2d_26 (Conv2D)          (None, 28, 28, 8)         584       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 14, 14, 8)        0         
 g2D)                                                     

In [106]:
encoder = keras.Model(input_img, encoded)
encoded_imgs = encoder.predict(X_train)
print(encoded_imgs.shape, X_train.shape)
batch_size = 32

train_dataset = MNIST_Dataset(encoded_imgs, y_train)

# Loading
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

(27000, 14, 14, 8) (27000, 56, 56)
(27000, 14, 14, 8)


In [ ]:
encoder = keras.Model(input_img, encoded)
encoded_imgs = encoder.predict(validation_unlabel)
print(encoded_imgs.shape)

n = 10
plt.figure(figsize=(20, 8))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(encoded_imgs[i].reshape((7, 7 * 8)).T)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [104]:
class ConvNet_4Layers(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super(ConvNet_4Layers, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=28, kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(28, 28, 2),
            nn.BatchNorm2d(28),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(dropout))
        self.layer2 = nn.Sequential(
            nn.Conv2d(28, 56, 2),
            nn.ReLU(),
            nn.Conv2d(56, 56, 1),
            nn.BatchNorm2d(56),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(dropout))
        self.layer3 = nn.Sequential(
            nn.Conv2d(56, 112, 1),
            nn.ReLU(),
            nn.Conv2d(112, 112, 1),
            nn.BatchNorm2d(112),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(dropout))
        self.layer4 = nn.Sequential(
            nn.Conv2d(112, 224, 1),
            nn.ReLU(),
            nn.Conv2d(224, 224, 1),
            nn.BatchNorm2d(224),
            nn.ReLU(),
            # nn.MaxPool2d(2, stride=2),
            nn.Dropout(dropout))
        self.fc = nn.Linear(224, output_dim)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
def predict(model, data_loader):
    real=[]
    for i, data in enumerate(data_loader):
        output = model(data)
        for i in range(output.shape[0]):
            com = onehot(output[i])
            real.append(com)   
    return torch.Tensor(real)

In [13]:
def onehot(vector):
    digit=vector[0:10]
    alphabet=vector[10:36]
    max_1=torch.argmax(digit,0,keepdim=True)
    max_2=torch.argmax(alphabet,0,keepdim=True)
    one_hot1=torch.FloatTensor(digit.shape)
    one_hot2=torch.FloatTensor(alphabet.shape)
    one_hot1.zero_()
    one_hot2.zero_()
    one_hot1.scatter_(0, max_1, 1)
    one_hot2.scatter_(0, max_2, 1)
    return torch.cat((one_hot1,one_hot2),0).tolist()

In [107]:

def train_model(model_class, validation_loader, y_validation, dropout=0.5, epochs=30, criterion=nn.CrossEntropyLoss(), learning_rate=1e-3):
    input_dim = 56 * 56
    output_dim = 36
    model = model_class(input_dim, output_dim, dropout)
    # encoder = keras.Model(input_img, encoded)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    losses_per_epoch = []
    validation_score_per_epoch = []
    for epoch in range(epochs):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
#             labels = labels.to(dtype=torch.long)
            optimizer.zero_grad()

            # inputs = encoder.predict(inputs)
            # print(type(inputs))
            outputs = model(inputs) #### 1:00
#             real=[]
#             for i in range(outputs.shape[0]):
#                 com = onehot(outputs[i])
#                 real.append(com.tolist())  
            loss = criterion(outputs, labels)#torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()
        
        model.eval()
       
        validation_predictions = predict(model, validation_loader).numpy()
#         y_val = np.argmax(y_validation, axis = 1)
       
        validation_score_per_epoch.append(accuracy_score(y_validation.tolist(), validation_predictions.tolist()))
        model.train()
        
        losses_per_epoch.append(loss)
    print('[Finished Training] Score: %.4f Loss: %.4f' % (validation_score_per_epoch[-1], losses_per_epoch[-1]))
    return model, losses_per_epoch, validation_score_per_epoch

In [108]:
#test
model, loss, score = train_model(ConvNet_4Layers, validation_loader, y_validation, dropout=0.1, epochs=10)

KeyboardInterrupt: 